Tabellskapande:

Två tabeller skapas:
VerifiedBackups lagrar information om backup-filer och datumet då de verifierades.
BackupLocations sparar servernamn och backup-sökvägar.

In [ ]:
CREATE TABLE VerifiedBackups (
    BackupFile NVARCHAR(255) PRIMARY KEY,
    VerificationDate DATETIME
)
CREATE TABLE BackupLocations (
    ServerName NVARCHAR(255),
    BackupPath NVARCHAR(255)
)


Infoga backup-platser:

Infoga servernamn och sökväg i tabellen BackupLocations.

In [ ]:
INSERT INTO BackupLocations (ServerName, BackupPath)
VALUES 
('Server1', '\\Server1\BackupFolder\'),
('Server2', '\\Server2\BackupFolder\'),
('Server3', '\\Server3\BackupFolder\')


Backup-verifieringsprocessen:

Deklarera variabler för servernamn, backup-sökväg, backup-fil och felmeddelanden.
En cursor itererar genom varje server och backup-sökväg, hämtar listan över .bak-filer med hjälp av xp_cmdshell, och kontrollerar om filen redan har verifierats.
Om en fil inte är verifierad, körs kommandot RESTORE VERIFYONLY för att kontrollera att backupen är giltig.
Om verifieringen misslyckas, skickas ett e-postmeddelande med detaljer om felet.

In [ ]:
DECLARE @ServerName NVARCHAR(255)
DECLARE @BackupPath NVARCHAR(255)
DECLARE @BackupFile NVARCHAR(255)
DECLARE @Command NVARCHAR(1000)
DECLARE @ErrorMessage NVARCHAR(4000)

-- Skapa en tabell för att lagra backupfiler
CREATE TABLE #BackupFiles (BackupFile NVARCHAR(255))

-- Iterera genom varje server och mapp
DECLARE LocationCursor CURSOR FOR
SELECT ServerName, BackupPath FROM BackupLocations

OPEN LocationCursor
FETCH NEXT FROM LocationCursor INTO @ServerName, @BackupPath

WHILE @@FETCH_STATUS = 0
BEGIN
    -- Fyll tabellen med backupfiler från nätverkskatalogen
    TRUNCATE TABLE #BackupFiles
    INSERT INTO #BackupFiles (BackupFile)
    EXEC ('xp_cmdshell ''dir /b ' + @BackupPath + '*.bak''')

    -- Iterera genom varje backupfil och kör RESTORE VERIFYONLY om den inte redan är verifierad
    DECLARE BackupCursor CURSOR FOR
    SELECT BackupFile FROM #BackupFiles

    OPEN BackupCursor
    FETCH NEXT FROM BackupCursor INTO @BackupFile

    WHILE @@FETCH_STATUS = 0
    BEGIN
        IF NOT EXISTS (SELECT 1 FROM VerifiedBackups WHERE BackupFile = @BackupFile)
        BEGIN
            BEGIN TRY
                SET @Command = N'RESTORE VERIFYONLY FROM DISK = ''' + @BackupPath + @BackupFile + ''''
                EXEC sp_executesql @Command

                -- Lägg till filen i VerifiedBackups-tabellen
                INSERT INTO VerifiedBackups (BackupFile, VerificationDate)
                VALUES (@BackupFile, GETDATE())
            END TRY
            BEGIN CATCH
                SET @ErrorMessage = ERROR_MESSAGE()
                EXEC msdb.dbo.sp_send_dbmail
                    @profile_name = 'Profil',
                    @recipients = 'andreas@roort.com',
                    @subject = 'Backup Verification Failed',
                    @body = 'Verification of backup file ' + @BackupFile + ' on server ' + @ServerName + ' failed. Error: ' + @ErrorMessage
            END CATCH
        END
        FETCH NEXT FROM BackupCursor INTO @BackupFile
    END

    CLOSE BackupCursor
    DEALLOCATE BackupCursor

    FETCH NEXT FROM LocationCursor INTO @ServerName, @BackupPath
END

CLOSE LocationCursor
DEALLOCATE LocationCursor

-- Rensa upp
DROP TABLE #BackupFiles


Todo:

. Använd SQL Server Agent för att schemalägga körningen av skriptet vid regelbundna intervaller (dagligen?). 

. Utöka skript för att skriva till en loggtabell som innehåller varje verifieringskörning, inklusive resultat, verifieringstid och eventuella fel.

. Vid nätverks- eller åtkomstproblem inträffar ska vi logga dessa och försöka igen efter tidsenhet(timme?) , istället för att bara skicka ett felmeddelande. Skicka felmeddelande efter 3 försök. 



Bonusfunktioner:

. Checksum-kontroller?: Använd CHECKSUM vid backup-skapandet och verifieringen för att ytterligare säkerställa att backup-filerna är intakta.

. Indexera BackupFile-kolumnen i tabellen VerifiedBackups för att snabba upp verifieringsprocessen.

. Integrera med Prometheus 

. Valbart om man ska ha verifyonly eller ej

. Valbart om man sparar till log, tabell eller skickar mail


Förslag från ChatGPT:

Användning av xp_cmdshell: Detta anses ofta vara en säkerhetsrisk eftersom det tillåter exekvering av kommandon på operativsystemnivå. xp_cmdshell bör endast vara aktiverat om det är absolut nödvändigt och måste användas med försiktighet.

Felfångst och hantering: Skriptet fångar fel och skickar e-postnotifikationer, men det skulle kunna förbättras med mer omfattande loggning för att spåra verifieringsförsök som misslyckas, särskilt om återförsök ska implementeras.

Prestandaoptimering: Användningen av cursors kan leda till prestandaproblem om det finns många servrar eller backup-filer att iterera igenom. En set-baserad lösning skulle kunna förbättra prestandan.

Loggning och övervakning: Integrationen med Prometheus är ett bra förslag, men en tydlig strategi för att övervaka långsiktiga trender, problem och flaskhalsar skulle vara fördelaktig.

Tabellstrukturen för BackupLocations: Det kan vara bra att lägga till unika restriktioner eller primärnycklar för kolumnerna ServerName och BackupPath för att undvika dubbletter.
